In [3]:
import numpy as np
import pandas as pd
import json
import collections
from collections import defaultdict 
from functools import partial
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm


In [7]:
mimic_dir = '/home/asem/GP/ehr-data/phys-mimic4v2.0/physionet.org/files/mimiciv/2.0'

In [8]:
# Good read: https://iq-inc.com/importerror-attempted-relative-import/

import sys
import importlib

### Notes

#### TODO

1. From DIAGNOSES_ICD:
    - map to CCS (multi/single)
    - histogram of diseases (number of patients for each CCS code)
2. From ADMISSION and ICUSTAY:
    - #Admissions per patient
    - #ICUStays per patient
    - #ICUStays per admission
3. measurements: merge all measurements by HADM_ID (aggregate by median, 0.9 quantile, 0.1 quantile).

In [10]:
admissions = pd.read_csv(f'{mimic_dir}/hosp/admissions.csv.gz')

In [11]:
admissions.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0


In [12]:
len(set(admissions.subject_id))

190279

In [13]:
patients_admission = defaultdict(set)

for row in admissions.itertuples():
    patients_admission[row.subject_id].add(row.hadm_id)
    
patients_admission_df = pd.DataFrame({'patient': patients_admission.keys(), 'n_admissions': map(len, patients_admission.values())})

In [14]:
patients_admission_count = list(map(lambda t: (t[0], len(t[1])), patients_admission.items()))

In [15]:
_, counts = zip(*patients_admission_count)

In [16]:
admissions_hist = np.bincount(counts)

In [17]:
admissions_hist

array([     0, 106468,  37638,  16792,   9190,   5491,   3536,   2415,
         1783,   1364,    991,    797,    626,    494,    419,    325,
          250,    198,    178,    147,    150,    101,    108,     73,
           71,     55,     52,     57,     40,     32,     39,     30,
           29,     30,     21,     29,     22,     19,     11,     11,
           14,     16,     10,      6,     14,      6,      6,      8,
            7,     11,      6,      5,      5,      7,      2,      6,
            4,      3,      5,      2,      2,      1,      2,      1,
            6,      1,      0,      2,      3,      3,      1,      3,
            1,      0,      0,      1,      1,      1,      1,      2,
            1,      0,      1,      0,      2,      1,      1,      0,
            1,      2,      0,      0,      2,      1,      1,      1,
            0,      1,      1,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [18]:
# In GRAM paper, all patients with at least 2 visits were utilized (n=7537).
np.sum(patients_admission_df.n_admissions > 1)

83811

### Select patients with at least 2 visits

- Followed by GRAM paper.
- n=7537

In [19]:
multi_visit_mimic_dir = '/home/asem/GP/ehr-data/mimic4-cohort'

patients_multi_visits = set(patients_admission_df[patients_admission_df.n_admissions > 1].patient.tolist())

In [20]:
patients = pd.read_csv(f'{mimic_dir}/hosp/patients.csv.gz')
patients = patients[patients.subject_id.isin(patients_multi_visits)].reset_index(drop=True)
patients.to_csv(f'{multi_visit_mimic_dir}/patients.csv.gz', compression='gzip', index=False)
patients.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000084,M,72,2160,2017 - 2019,2161-02-13
2,10000117,F,48,2174,2008 - 2010,NaN
3,10000826,F,32,2146,2008 - 2010,NaN
4,10000883,M,20,2124,2008 - 2010,NaN


In [21]:
patients = pd.read_csv(f'{multi_visit_mimic_dir}/patients.csv.gz')
patients.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000084,M,72,2160,2017 - 2019,2161-02-13
2,10000117,F,48,2174,2008 - 2010,NaN
3,10000826,F,32,2146,2008 - 2010,NaN
4,10000883,M,20,2124,2008 - 2010,NaN


In [22]:
admissions = admissions[admissions.subject_id.isin(patients_multi_visits)].reset_index(drop=True)
admissions.to_csv(f'{multi_visit_mimic_dir}/admissions.csv.gz', compression='gzip', index=False)


In [24]:
diagnoses_icd = pd.read_csv(f'{mimic_dir}/hosp/diagnoses_icd.csv.gz', dtype = {'icd_code': str})
diagnoses_icd = diagnoses_icd[diagnoses_icd.subject_id.isin(patients_multi_visits)].reset_index(drop=True)
diagnoses_icd.to_csv(f'{multi_visit_mimic_dir}/diagnoses_icd.csv.gz', compression='gzip', index=False)


procedures_icd = pd.read_csv(f'{mimic_dir}/hosp/procedures_icd.csv.gz', dtype = {'icd_code': str})
procedures_icd = procedures_icd[procedures_icd.subject_id.isin(patients_multi_visits)].reset_index(drop=True)
procedures_icd.to_csv(f'{multi_visit_mimic_dir}/procedures_icd.csv.gz', compression='gzip', index=False)

